In [7]:
import pandas as pd

In [8]:
import pandas as pd
import numpy as np

# Load your CSV data (or replace this with your actual DataFrame)
df = pd.read_csv("ligaacb-eurocup-2324.csv")

# Get list of unique teams
teams = sorted(set(df['Team']).union(set(df['Opp'])))
team_indices = {team: i for i, team in enumerate(teams)}
n = len(teams)

# Initialize Colley matrix and b vector
C = 2 * np.eye(n) # diagonal initalized to 2 and incremented by games played, other entries will be updated to negative games played
b = np.zeros(n) # later will be updated to 1 + (W-L)/2

# Loop over each match
for _, row in df.iterrows():
    team = row['Team']
    opp = row['Opp']
    pts_team = row['PTS']
    pts_opp = row['PTS_Opp']
    weight = row.get('Weight', 1.0)

    i = team_indices[team]
    j = team_indices[opp]

    # Update matrix
    C[i, i] += weight
    C[j, j] += weight
    C[i, j] -= weight
    C[j, i] -= weight

    # Update b vector with W-L record
    if pts_team > pts_opp:
        b[i] += weight
        b[j] -= weight
    elif pts_team < pts_opp:
        b[i] -= weight
        b[j] += weight
    else: 
        b[i] += weight / 2
        b[j] += weight / 2
   
# Adjust b vector to be 1 + (W-L)/2
for i in range(n):
    b[i] = 1 + (b[i] / 2)

# Solve the system
r = np.linalg.solve(C, b)

spanish_teams = set(pd.read_csv("ligaacb-2324.csv")["Team"])
ranking_increment = 1
# Output rankings
rankings = sorted(zip(teams, r), key=lambda x: x[1], reverse=True)
for rank, (team, score) in enumerate(rankings, 1):
    if team in spanish_teams:
        print(f"{ranking_increment}. {team}: {score:.4f}")
        ranking_increment += 1

1. Unicaja: 0.8332
2. Real Madrid: 0.8285
3. Barça: 0.7062
4. Lenovo Tenerife: 0.6666
5. UCAM Murcia: 0.6576
6. Valencia Basket: 0.6562
7. Dreamland Gran Canaria: 0.6312
8. Baxi Manresa: 0.6062
9. Joventut: 0.6060
10. Baskonia: 0.5812
11. MoraBanc Andorra: 0.4713
12. Casademont Zaragoza: 0.4672
13. Bàsquet Girona: 0.4562
14. Surne Bilbao Basket: 0.4562
15. Monbus Obradoiro: 0.4062
16. Covirán Granada: 0.4062
17. Río Breogán: 0.4062
18. Zunder Palencia: 0.2812
